In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from matplotlib.ticker import FormatStrFormatter

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error

In [2]:
df=pd.read_csv('data/df_out.csv')
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,NaN,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021.0,nissan,altima,gas,8000,clean,automatic,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,ny,40.654800,-73.609700,2021-05-03T18:32:06-0400,medium,2021-05-03 22:32:06+00:00


# Data Preparation


Целевой переменной является price_category, найдём в каких долях входят значения в выборку

In [3]:
list_name_category = df.price_category.unique()
list_quantity_category = df.price_category.value_counts()
summ = df.price_category.count()

for i in range(len(list_name_category)):
    print(f'Доля машин в категории {list_name_category[i]} составляет {list_quantity_category[i]/summ}')

Доля машин в категории high составляет 0.3495
Доля машин в категории medium составляет 0.328
Доля машин в категории low составляет 0.3225


## Data Cleaning


Информация о датафрейме

In [4]:
df.describe(include = 'all')

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
count,1.000000e+04,10000,10000,10000,1.000000e+04,9964.000000,10000,9872,9937,10000.00000,9834,9955,9998,9998,10000,9902.000000,9902.000000,9998,10000,9998
unique,NaN,10000,393,405,NaN,NaN,40,3466,5,NaN,6,3,9081,9845,51,NaN,NaN,9957,3,9911
top,NaN,https://chattanooga.craigslist.org/ctd/d/chatt...,"kansas city, MO",https://kansascity.craigslist.org,NaN,NaN,ford,f-150,gas,NaN,clean,automatic,https://images.craigslist.org/00N0N_1xMPvfxRAI...,PLEASE VIEW THE FREE CARFAX HISTORY REPORT AT ...,ca,NaN,NaN,2021-04-19T10:00:54-0400,high,2021-04-22 15:01:24+00:00
freq,NaN,1,86,86,NaN,NaN,1638,151,8402,NaN,9518,7700,188,7,1108,NaN,NaN,2,3495,3
mean,7.311541e+09,NaN,NaN,NaN,2.067842e+04,2010.927941,NaN,NaN,NaN,93324.13400,NaN,NaN,NaN,NaN,NaN,38.594142,-94.165877,NaN,NaN,NaN
std,4.477653e+06,NaN,NaN,NaN,1.246230e+05,9.672040,NaN,NaN,NaN,63181.17499,NaN,NaN,NaN,NaN,NaN,5.844895,18.128504,NaN,NaN,NaN
min,7.208550e+09,NaN,NaN,NaN,5.000000e+02,1915.000000,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,NaN,NaN,-67.144243,-158.069300,NaN,NaN,NaN
25%,7.308189e+09,NaN,NaN,NaN,7.900000e+03,2008.000000,NaN,NaN,NaN,39147.25000,NaN,NaN,NaN,NaN,NaN,34.830368,-110.322086,NaN,NaN,NaN
50%,7.312757e+09,NaN,NaN,NaN,1.574950e+04,2013.000000,NaN,NaN,NaN,89000.00000,NaN,NaN,NaN,NaN,NaN,39.298522,-88.000050,NaN,NaN,NaN
75%,7.315274e+09,NaN,NaN,NaN,2.799000e+04,2017.000000,NaN,NaN,NaN,136875.00000,NaN,NaN,NaN,NaN,NaN,42.428070,-80.830000,NaN,NaN,NaN


Распечатаем в цикле по каждой колонке название колонки, количество уникальных значений, а затем список возможных значений вместе с их количеством появления в датасете.

In [5]:
for i in list(df):
    print(f'Характеристика: {i}')
    print(f'Количество уникальных значений: {df[i].nunique()}')
    print(f'Список значений: {df[i].value_counts()}')
    print('----------------------------------------')

Характеристика: id
Количество уникальных значений: 10000
Список значений: 7308295377    1
7316908621    1
7305833526    1
7316719384    1
7314440026    1
             ..
7305589280    1
7314411334    1
7307782698    1
7305083080    1
7311960763    1
Name: id, Length: 10000, dtype: int64
----------------------------------------
Характеристика: url
Количество уникальных значений: 10000
Список значений: https://chattanooga.craigslist.org/ctd/d/chattanooga-2020-ram-2500-crew-cab-big/7308295377.html     1
https://boise.craigslist.org/ctd/d/boise-2015-volkswagen-jetta-18t-sport/7316908621.html            1
https://appleton.craigslist.org/cto/d/waupaca-1932-ford-replica-window-with/7305833526.html         1
https://goldcountry.craigslist.org/ctd/d/placerville-2017-chevy-chevrolet/7316719384.html           1
https://roanoke.craigslist.org/ctd/d/roanoke-2012-chevrolet-chevy-impala/7314440026.html            1
                                                                                      

Количество полностью заполненных объектов и процент пропущенных значений

In [6]:
missing_values_out_clean = ((df.isna().sum() / len(df))*100).sort_values()[::-1]
print('Процент пропущенных значений: ')
missing_values_out_clean

Процент пропущенных значений: 


title_status      1.66
model             1.28
long              0.98
lat               0.98
fuel              0.63
transmission      0.45
year              0.36
date              0.02
posting_date      0.02
description       0.02
image_url         0.02
price             0.00
url               0.00
region            0.00
region_url        0.00
odometer          0.00
manufacturer      0.00
price_category    0.00
state             0.00
id                0.00
dtype: float64

Колонки, в которых имеются пропущенные значения

In [7]:
missing_colomns = missing_values_out_clean[missing_values_out_clean.values > 0].index.tolist()
missing_colomns

['title_status',
 'model',
 'long',
 'lat',
 'fuel',
 'transmission',
 'year',
 'date',
 'posting_date',
 'description',
 'image_url']

### Обработка пропусков


#### Обработка пропусков в годе выпуска автомобиля(year)

Количество пропущенных значений в годе выпуска автомобиля

In [10]:
quant_y = df.year.isna().sum()
print(f"Количество пропущенных значений в year:{quant_y}")

Количество пропущенных значений в year:36


Записи, в которых год выпуска автомобиля не заполнен

In [8]:
df[df.year.isna()]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
344,7304102133,https://wenatchee.craigslist.org/ctd/d/wenatch...,wenatchee,https://wenatchee.craigslist.org,35880,NaN,other,500,gas,13508,clean,automatic,https://images.craigslist.org/00J0J_6IMoEUzIIM...,2019 *Ram* *1500* Big Horn/Lone Star 4x2 Quad ...,wa,47.434493,-120.320759,2021-04-09T06:31:03-0700,high,2021-04-09 13:31:03+00:00
420,7316337069,https://lasvegas.craigslist.org/ctd/d/scottsda...,las vegas,https://lasvegas.craigslist.org,84995,NaN,other,Super Duty F-350 SRW,diesel,1268,clean,automatic,https://images.craigslist.org/00x0x_iQCC0Ef28S...,2020 *Ford* *Super Duty F-350 SRW* 1K MILES. B...,nv,33.458277,-111.925970,2021-05-03T11:31:39-0700,high,2021-05-03 18:31:39+00:00
425,7315662117,https://westslope.craigslist.org/ctd/d/denver-...,western slope,https://westslope.craigslist.org,43900,NaN,other,Grand Caravan,NaN,21,clean,automatic,https://images.craigslist.org/00f0f_7AwipNocNH...,"2019 *Dodge* *Grand Caravan* SE - $43,900Call ...",co,39.674406,-104.998604,2021-05-02T01:11:33-0600,high,2021-05-02 07:11:33+00:00
702,7316199009,https://nashville.craigslist.org/ctd/d/nashvil...,nashville,https://nashville.craigslist.org,28498,NaN,other,olet Silverado 1500,NaN,59246,NaN,automatic,https://images.craigslist.org/00T0T_dZ7nOXskCS...,2019 *Chevrolet* *Silverado 1500* 2019 CHEVROL...,tn,36.138022,-86.732261,2021-05-03T10:00:40-0500,high,2021-05-03 15:00:40+00:00
893,7316840794,https://denver.craigslist.org/ctd/d/american-f...,denver,https://denver.craigslist.org,60900,NaN,other,F-150,gas,6110,clean,automatic,https://images.craigslist.org/01616_bpzSdSRTbx...,"2020 *Ford* *F-150* PLATINUM FX4 Truck - $60,9...",co,40.362248,-111.785947,2021-05-04T12:11:22-0600,high,2021-05-04 18:11:22+00:00
906,7311915616,https://westslope.craigslist.org/ctd/d/denver-...,western slope,https://westslope.craigslist.org,67900,NaN,other,Odyssey,gas,21,clean,automatic,https://images.craigslist.org/00404_erlbeKWEkn...,2020 *Honda* *Odyssey* EX-L w/Navi/RES Automat...,co,39.674406,-104.998604,2021-04-24T10:41:06-0600,high,2021-04-24 16:41:06+00:00
1005,7306735187,https://westslope.craigslist.org/ctd/d/denver-...,western slope,https://westslope.craigslist.org,38900,NaN,other,Grand Caravan,NaN,21,clean,automatic,https://images.craigslist.org/00F0F_3T6FM0XMZW...,"2019 *Dodge* *Grand Caravan* SE - $38,900Call ...",co,39.674406,-104.998604,2021-04-14T10:51:36-0600,high,2021-04-14 16:51:36+00:00
1217,7309954650,https://gainesville.craigslist.org/ctd/d/gaine...,gainesville,https://gainesville.craigslist.org,24975,NaN,other,CR-V,gas,44128,clean,automatic,https://images.craigslist.org/00000_2TUtAgLlCR...,"2019 *Honda* *CR-V* EX 2WD SUV - $24,975Call o...",fl,29.676827,-82.319202,2021-04-20T14:46:55-0400,high,2021-04-20 18:46:55+00:00
1319,7313865045,https://westslope.craigslist.org/ctd/d/denver-...,western slope,https://westslope.craigslist.org,47900,NaN,other,a Sienna,gas,43701,clean,automatic,https://images.craigslist.org/00o0o_8XYp4Zygdj...,2018 *Toyota* *Sienna* LE Automatic Access Sea...,co,39.674406,-104.998604,2021-04-28T10:12:14-0600,high,2021-04-28 16:12:14+00:00
1845,7303344554,https://daytona.craigslist.org/ctd/d/new-smyrn...,daytona beach,https://daytona.craigslist.org,35850,NaN,other,Transit Cargo Van,gas,567,clean,automatic,https://images.craigslist.org/00a0a_d4XVrHnlxl...,"2020 *Ford* *Transit Cargo Van* T-250 148"" WB*...",fl,29.046240,-80.934561,2021-04-07T17:10:06-0400,high,2021-04-07 21:10:06+00:00


Удаляем записи, в которых год выпуска автомобиля не заполнен, и сохраняем обновлённый датафрейм.

In [9]:
df = df.loc[~df.year.isna()]
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,NaN,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021.0,nissan,altima,gas,8000,clean,automatic,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,ny,40.654800,-73.609700,2021-05-03T18:32:06-0400,medium,2021-05-03 22:32:06+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002.0,toyota,rav4,gas,150000,clean,automatic,https://images.craigslist.org/00n0n_aaEBqmzz5Z...,Transmission:Automatic Exterior Color:WHITE In...,ny,42.123900,-79.189500,2021-04-10T16:33:57-0400,low,2021-04-10 20:33:57+00:00
9996,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008.0,jeep,wrangler,gas,113573,clean,other,https://images.craigslist.org/00000_9ZNVdXaiei...,2008 Jeep Wrangler X Offered by: R&L Auto -- ...,ny,43.216990,-77.755610,2021-05-03T09:36:30-0400,medium,2021-05-03 13:36:30+00:00
9997,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011.0,audi,a3 2.0t premium plus pzev,gas,150184,clean,automatic,https://images.craigslist.org/00X0X_k5JiZDDyqM...,2011 Audi A3 2.0T Premium Plus PZEV Wagon2011 ...,or,44.925908,-122.982753,2021-04-22T12:14:01-0700,low,2021-04-22 19:14:01+00:00
9998,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015.0,porsche,cayenne,hybrid,61943,clean,automatic,https://images.craigslist.org/00O0O_gBihGEQxYT...,"2015 Porsche Cayenne AWD 4dr S E-Hybrid - $31,...",wi,43.029559,-89.397796,2021-04-14T09:14:42-0500,high,2021-04-14 14:14:42+00:00


 Проверяем количество пропущенных значений в годе выпуска автомобиля для обновлённого датафрейма.

In [10]:
quant = df.year.isna().sum()
print(f"Количество пропущенных значений в year:{quant}")

Количество пропущенных значений в year:0


#### Обработка пропусков в типе топлива(fuel)

---



Количество значений для типа топлива, включая пропущенные значения

In [11]:
 df.fuel.value_counts(dropna = False)

gas         8379
other        729
diesel       650
hybrid       108
NaN           54
electric      44
Name: fuel, dtype: int64

Записи, в которых год выпуска автомобиля не заполнен

In [12]:
df[df.fuel.isna()]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,NaN,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
32,7309910590,https://eugene.craigslist.org/ctd/d/eugene-200...,eugene,https://eugene.craigslist.org,4977,2001.0,toyota,camry solara 2dr cpe se v6 auto (natl),NaN,193242,clean,automatic,https://images.craigslist.org/00F0F_ebtJlxCQjv...,***Call Us for more information at: 541-228-94...,or,44.076513,-123.150156,2021-04-20T10:40:15-0700,low,2021-04-20 17:40:15+00:00
78,7314559074,https://eugene.craigslist.org/ctd/d/eugene-200...,eugene,https://eugene.craigslist.org,27977,2009.0,ford,"f-150 4wd supercrew 145"" platinum",NaN,79522,clean,automatic,https://images.craigslist.org/00L0L_khoMmqXaSJ...,***Call Us for more information at: 541-228-94...,or,44.076513,-123.150156,2021-04-29T15:15:30-0700,high,2021-04-29 22:15:30+00:00
275,7315141987,https://honolulu.craigslist.org/oah/ctd/d/fort...,hawaii,https://honolulu.craigslist.org,29888,2019.0,nissan,frontier crew cab 4x2 sv auto,NaN,10679,clean,automatic,https://images.craigslist.org/00F0F_d9wEXQuSyu...,***Call Us for more information at: 877-262-34...,hi,21.344376,-157.896975,2021-04-30T15:50:19-1000,high,2021-05-01 01:50:19+00:00
471,7316807613,https://eugene.craigslist.org/ctd/d/eugene-200...,eugene,https://eugene.craigslist.org,13436,2006.0,chevrolet,"silverado 1500 ext cab 143.5"" wb 4wd ls",NaN,172700,clean,automatic,https://images.craigslist.org/00l0l_35tV0gQdD8...,***Call Us for more information at: 541-228-94...,or,44.076513,-123.150156,2021-05-04T10:20:19-0700,medium,2021-05-04 17:20:19+00:00
1009,7314561306,https://cnj.craigslist.org/ctd/d/parlin-2000-c...,central NJ,https://cnj.craigslist.org,5894,2000.0,chevrolet,"s-10 ext cab 123"" wb ls",NaN,78740,clean,automatic,https://images.craigslist.org/00O0O_8vd70bT1rY...,***Call Us for more information at: 732-316-55...,nj,40.468650,-74.296990,2021-04-29T18:20:19-0400,low,2021-04-29 22:20:19+00:00
1279,7310297136,https://westernmass.craigslist.org/ctd/d/johns...,western massachusetts,https://westernmass.craigslist.org,6000,2008.0,other,IC IC,NaN,0,NaN,automatic,https://images.craigslist.org/00404_cBFpBKbfr2...,"2008 *IC* *IC* CE300 - $6,000Call Us Today! 85...",ma,43.008105,-74.352011,2021-04-21T10:11:05-0400,low,2021-04-21 14:11:05+00:00
1318,7310414739,https://nwct.craigslist.org/ctd/d/melville-200...,northwest CT,https://nwct.craigslist.org,11995,2007.0,gmc,c-7500 multiple uses perfect,NaN,50098,clean,automatic,https://images.craigslist.org/00T0T_jvsOAKSGAr...,2007 *GMC* *C-7500 MULTIPLE USES PERFECT* FOR ...,ct,40.681179,-73.430488,2021-04-21T13:18:41-0400,medium,2021-04-21 17:18:41+00:00
1686,7316342709,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,19942,2017.0,ram,"promaster cargo van 1500 low roof 118"" wb",NaN,88836,clean,automatic,https://images.craigslist.org/00707_6ZlIIpYmGf...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T14:40:24-0400,medium,2021-05-03 18:40:24+00:00
1847,7315151536,https://honolulu.craigslist.org/oah/ctd/d/fort...,hawaii,https://honolulu.craigslist.org,29988,2019.0,toyota,sienna le fwd 8-passenger (natl),NaN,18762,clean,automatic,https://images.craigslist.org/00R0R_dO0gtiRwLa...,***Call Us for more information at: 877-262-34...,hi,21.344376,-157.896975,2021-04-30T16:25:22-1000,high,2021-05-01 02:25:22+00:00


Заполним незаполненные значения типа топлива значением other

In [13]:
df.fuel = df.fuel.fillna('other')
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,other,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021.0,nissan,altima,gas,8000,clean,automatic,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,ny,40.654800,-73.609700,2021-05-03T18:32:06-0400,medium,2021-05-03 22:32:06+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002.0,toyota,rav4,gas,150000,clean,automatic,https://images.craigslist.org/00n0n_aaEBqmzz5Z...,Transmission:Automatic Exterior Color:WHITE In...,ny,42.123900,-79.189500,2021-04-10T16:33:57-0400,low,2021-04-10 20:33:57+00:00
9996,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008.0,jeep,wrangler,gas,113573,clean,other,https://images.craigslist.org/00000_9ZNVdXaiei...,2008 Jeep Wrangler X Offered by: R&L Auto -- ...,ny,43.216990,-77.755610,2021-05-03T09:36:30-0400,medium,2021-05-03 13:36:30+00:00
9997,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011.0,audi,a3 2.0t premium plus pzev,gas,150184,clean,automatic,https://images.craigslist.org/00X0X_k5JiZDDyqM...,2011 Audi A3 2.0T Premium Plus PZEV Wagon2011 ...,or,44.925908,-122.982753,2021-04-22T12:14:01-0700,low,2021-04-22 19:14:01+00:00
9998,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015.0,porsche,cayenne,hybrid,61943,clean,automatic,https://images.craigslist.org/00O0O_gBihGEQxYT...,"2015 Porsche Cayenne AWD 4dr S E-Hybrid - $31,...",wi,43.029559,-89.397796,2021-04-14T09:14:42-0500,high,2021-04-14 14:14:42+00:00


 Проверяем количество пропущенных значений в типе топлива для обновлённого датафрейма.

In [14]:
quant = df.fuel.isna().sum()
print(f"Количество пропущенных значений в fuel:{quant}")

Количество пропущенных значений в fuel:0


#### Обработка пропусков в модели автомобиля(model)

Количество значений для модели автомобиля, включая пропущенные значения

In [15]:
 df.model.value_counts(dropna = False)

f-150                        151
NaN                          126
silverado 1500               110
1500                         100
camry                         77
                            ... 
sorrento lx v6                 1
sierra1500 crewcab 4x4         1
corolla se                     1
xc60 t6 inscription sport      1
a3 2.0t premium plus pzev      1
Name: model, Length: 3447, dtype: int64

Записи, в которых год выпуска автомобиля не заполнен

In [16]:
df[df.model.isna()]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
67,7316913641,https://ventura.craigslist.org/cto/d/camarillo...,ventura county,https://ventura.craigslist.org,3000,2005.0,rover,NaN,gas,165206,clean,automatic,https://images.craigslist.org/00S0S_b3yx6L4N2S...,2005 Range Rover HSE Changed Oil and now engi...,ca,34.219565,-119.001986,2021-05-04T13:11:15-0700,low,2021-05-04 20:11:15+00:00
142,7316550853,https://chicago.craigslist.org/chc/cto/d/midlo...,chicago,https://chicago.craigslist.org,3000,2006.0,jeep,NaN,gas,132000,clean,automatic,https://images.craigslist.org/01111_kNv3EKjH4H...,SUPER CLEAN 2006 JEEP LIBEERY RANAGADE 4X4 O...,il,41.618957,-87.726334,2021-05-03T20:33:40-0500,low,2021-05-04 01:33:40+00:00
151,7303575863,https://worcester.craigslist.org/ctd/d/fitchbu...,worcester / central MA,https://worcester.craigslist.org,7800,2007.0,gmc,NaN,gas,190000,clean,automatic,https://images.craigslist.org/00f0f_iWjiToqea2...,2007 gmc with plow leather loaded nice truck r...,ma,42.579600,-71.803100,2021-04-08T09:10:07-0400,low,2021-04-08 13:10:07+00:00
200,7315267860,https://gainesville.craigslist.org/ctd/d/kissi...,gainesville,https://gainesville.craigslist.org,19200,2012.0,ram,NaN,gas,121371,clean,automatic,https://images.craigslist.org/00W0W_hHV4D0Po0G...,2012 RAM 1500 Laramie Crew Cab 2WD Offered...,fl,28.329026,-81.404237,2021-05-01T10:05:17-0400,medium,2021-05-01 14:05:17+00:00
251,7316024407,https://reading.craigslist.org/ctd/d/paterson-...,reading,https://reading.craigslist.org,18695,2007.0,ram,NaN,diesel,220019,clean,automatic,https://images.craigslist.org/00n0n_jgq4eOvH7I...,2007 Dodge Ram 2500 SLT 4x4 Quad Cab Cummins D...,pa,40.894535,-74.152367,2021-05-02T21:06:09-0400,medium,2021-05-03 01:06:09+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,7314341390,https://desmoines.craigslist.org/cto/d/urbanda...,des moines,https://desmoines.craigslist.org,8000,2007.0,toyota,NaN,gas,85285,clean,automatic,https://images.craigslist.org/00K0K_aMoJgOIueB...,Toyota has been owned by non smoker with no pe...,ia,41.629500,-93.723000,2021-04-29T10:50:54-0500,low,2021-04-29 15:50:54+00:00
9616,7304358131,https://vermont.craigslist.org/ctd/d/salem-201...,vermont,https://vermont.craigslist.org,19944,2012.0,ram,NaN,gas,73082,clean,automatic,https://images.craigslist.org/00A0A_bfCaGJ20IX...,2012 RAM 1500 4WD REGULAR CAB SHORT BED 5.7 HE...,vt,42.788204,-71.233728,2021-04-09T16:24:23-0400,medium,2021-04-09 20:24:23+00:00
9670,7309686008,https://louisville.craigslist.org/cto/d/radcli...,louisville,https://louisville.craigslist.org,1600,2005.0,gmc,NaN,gas,52536,clean,automatic,https://images.craigslist.org/01111_99unV6cYER...,GMC Savana PRO AWD cargo van has been driven l...,ky,37.858900,-86.006900,2021-04-20T04:48:30-0400,low,2021-04-20 08:48:30+00:00
9926,7316447901,https://tulsa.craigslist.org/cto/d/tulsa-2017-...,tulsa,https://tulsa.craigslist.org,65000,2017.0,rover,NaN,gas,47000,clean,automatic,https://images.craigslist.org/00s0s_iGpmplDL99...,"2017 Range Rover HSE, supercharged V8, silver ...",ok,36.126200,-95.940800,2021-05-03T16:37:36-0500,high,2021-05-03 21:37:36+00:00


Заполним незаполненные значения типа топлива значением other

In [17]:
df.model = df.model.fillna('other')
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,other,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021.0,nissan,altima,gas,8000,clean,automatic,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,ny,40.654800,-73.609700,2021-05-03T18:32:06-0400,medium,2021-05-03 22:32:06+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002.0,toyota,rav4,gas,150000,clean,automatic,https://images.craigslist.org/00n0n_aaEBqmzz5Z...,Transmission:Automatic Exterior Color:WHITE In...,ny,42.123900,-79.189500,2021-04-10T16:33:57-0400,low,2021-04-10 20:33:57+00:00
9996,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008.0,jeep,wrangler,gas,113573,clean,other,https://images.craigslist.org/00000_9ZNVdXaiei...,2008 Jeep Wrangler X Offered by: R&L Auto -- ...,ny,43.216990,-77.755610,2021-05-03T09:36:30-0400,medium,2021-05-03 13:36:30+00:00
9997,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011.0,audi,a3 2.0t premium plus pzev,gas,150184,clean,automatic,https://images.craigslist.org/00X0X_k5JiZDDyqM...,2011 Audi A3 2.0T Premium Plus PZEV Wagon2011 ...,or,44.925908,-122.982753,2021-04-22T12:14:01-0700,low,2021-04-22 19:14:01+00:00
9998,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015.0,porsche,cayenne,hybrid,61943,clean,automatic,https://images.craigslist.org/00O0O_gBihGEQxYT...,"2015 Porsche Cayenne AWD 4dr S E-Hybrid - $31,...",wi,43.029559,-89.397796,2021-04-14T09:14:42-0500,high,2021-04-14 14:14:42+00:00


 Проверяем количество пропущенных значений в типе топлива для обновлённого датафрейма.

In [18]:
quant = df.model.isna().sum()
print(f"Количество пропущенных значений в model:{quant}")

Количество пропущенных значений в model:0


#### Обработка пропусков в статусе автомобиля(title_status)




Количество значений для статуса автомобиля, включая пропущенные значения.

In [19]:
 df.title_status.value_counts(dropna = False)

clean         9492
rebuilt        168
NaN            159
salvage         92
lien            35
missing         16
parts only       2
Name: title_status, dtype: int64

Описательные статистики для модели автомобиля.



In [20]:
df.title_status.describe(include = 'all')

count      9805
unique        6
top       clean
freq       9492
Name: title_status, dtype: object

Записи, в которых статус автомобиля не заполнен


In [21]:
df[df.title_status.isna()]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
11,7316540706,https://cincinnati.craigslist.org/ctd/d/fairfi...,cincinnati,https://cincinnati.craigslist.org,4995,2012.0,fiat,500 pop,gas,59242,NaN,manual,https://images.craigslist.org/00Z0Z_jbUmVcYlwR...,"2012 Fiat 500 Pop FOR ONLY $102/mo!59,242 mi. ...",oh,39.297045,-84.563243,2021-05-03T21:03:32-0400,low,2021-05-04 01:03:32+00:00
187,7313570594,https://orangecounty.craigslist.org/ctd/d/coro...,orange county,https://orangecounty.craigslist.org,17900,2013.0,ram,1500 tradesman quad cab,gas,145499,NaN,automatic,https://images.craigslist.org/00H0H_1JFMPCYyW8...,2013 Ram 1500 Tradesman Quad Cab PRICED TO SEL...,ca,33.873436,-117.542806,2021-04-27T15:03:47-0700,medium,2021-04-27 22:03:47+00:00
245,7315211652,https://ocala.craigslist.org/ctd/d/riverview-2...,ocala,https://ocala.craigslist.org,40990,2019.0,toyota,sienna xle,gas,3700,NaN,automatic,https://images.craigslist.org/01111_grKNF8GrXz...,2019 Toyota Sienna - Wheelchair Accessible Han...,fl,27.849760,-82.364950,2021-05-01T06:35:56-0400,high,2021-05-01 10:35:56+00:00
286,7306702976,https://toledo.craigslist.org/ctd/d/chelsea-73...,toledo,https://toledo.craigslist.org,4592,2003.0,chevrolet,avalanche 1500,gas,268691,NaN,automatic,https://images.craigslist.org/00808_1IOMJuA4GL...,$73/mo - 2003 Chevrolet Avalanche 1500 BaseCre...,oh,42.300673,-84.024108,2021-04-14T12:03:01-0400,low,2021-04-14 16:03:01+00:00
312,7307609203,https://roseburg.craigslist.org/ctd/d/eugene-2...,roseburg,https://roseburg.craigslist.org,18450,2017.0,ford,mustang,gas,66049,NaN,automatic,https://images.craigslist.org/00505_4apF36sLbP...,"2017 Ford MUSTANG - $18,450 \tMayberry Motors ...",or,44.058499,-123.106176,2021-04-16T04:06:29-0700,medium,2021-04-16 11:06:29+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9740,7301707830,https://tricities.craigslist.org/ctd/d/bristol...,tri-cities,https://tricities.craigslist.org,8995,2013.0,honda,odyssey,gas,203112,NaN,automatic,https://images.craigslist.org/00u0u_bawW7HN8xW...,"2013 *Honda* *Odyssey* 5dr EX Van FWD - $8,995...",tn,36.541821,-82.234645,2021-04-04T13:12:27-0400,low,2021-04-04 17:12:27+00:00
9805,7313949726,https://harrisburg.craigslist.org/ctd/d/brockp...,harrisburg,https://harrisburg.craigslist.org,8900,2012.0,honda,accord,other,95674,NaN,NaN,https://images.craigslist.org/01212_dsfYRNhxfd...,Car Starz Call Sales at 717-401-4221 2012 Hond...,pa,41.261540,-78.704520,2021-04-28T14:25:32-0400,low,2021-04-28 18:25:32+00:00
9840,7314987181,https://nashville.craigslist.org/ctd/d/nashvil...,nashville,https://nashville.craigslist.org,20900,2014.0,lexus,rx 350,gas,70800,NaN,automatic,https://images.craigslist.org/00Q0Q_cGablAkoYV...,2014 *Lexus* *RX 350* 2014 LEXUS RX 350 SUV 1-...,tn,36.138022,-86.732261,2021-04-30T15:00:48-0500,medium,2021-04-30 20:00:48+00:00
9856,7309717260,https://savannah.craigslist.org/ctd/d/jacksonv...,savannah / hinesville,https://savannah.craigslist.org,11495,2006.0,lexus,gx 470,gas,242000,NaN,automatic,https://images.craigslist.org/00s0s_8xg4gw6aAg...,"2006 *Lexus* *GX 470* 4dr SUV 4WD - $11,495By ...",ga,30.513566,-81.621875,2021-04-20T08:01:49-0400,medium,2021-04-20 12:01:49+00:00


Заполним незаполненные значения статуса автомобиля самым популярным значением, т.е. модой


In [22]:
df.title_status = df.title_status.fillna(df.title_status.mode)

Проверим для обновлённого датафрейма количество пропущенных значений для статуса автомобиля.


In [23]:
quant = df.title_status.isna().sum()
print(f"Количество пропущенных значений в title_status:{quant}")

Количество пропущенных значений в title_status:0


#### Обработка пропусков в коробке передач(transmission)

Количество значений для коробки передач, включая пропущенные значения

In [24]:
 df.transmission.value_counts(dropna = False)

automatic    7667
other        1623
manual        631
NaN            43
Name: transmission, dtype: int64

Записи, в которых год выпуска автомобиля не заполнен

In [25]:
df[df.transmission.isna()]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
111,7305109580,https://grandrapids.craigslist.org/ctd/d/comst...,grand rapids,https://grandrapids.craigslist.org,12750,2013.0,lexus,es,gas,109382,clean,NaN,https://images.craigslist.org/00n0n_kACT1l7OCN...,2013 Lexus ES 350 4dr Sdn Offered by: Auto...,mi,43.059210,-85.688970,2021-04-11T09:41:58-0400,medium,2021-04-11 13:41:58+00:00
112,7314951119,https://southjersey.craigslist.org/ctd/d/chest...,south jersey,https://southjersey.craigslist.org,18199,2012.0,jeep,wrangler,other,130174,clean,NaN,https://images.craigslist.org/00I0I_gRmuXmqil1...,2012 Jeep Wrangler Unlimited Sport 4WD Off...,nj,40.135190,-74.703570,2021-04-30T14:58:56-0400,medium,2021-04-30 18:58:56+00:00
817,7308021604,https://southjersey.craigslist.org/ctd/d/maple...,south jersey,https://southjersey.craigslist.org,11999,2015.0,honda,civic,gas,79797,clean,NaN,https://images.craigslist.org/00b0b_aCqgRZ7sAx...,2015 Honda Civic EX Offered by: VIP Auto O...,nj,39.938023,-74.971461,2021-04-16T19:46:55-0400,medium,2021-04-16 23:46:55+00:00
931,7314781483,https://nh.craigslist.org/ctd/d/plaistow-2018-...,new hampshire,https://nh.craigslist.org,64990,2018.0,ram,3500,other,8853,clean,NaN,https://images.craigslist.org/00000_iHpXs2BVmN...,2018 RAM Ram Chassis 3500 Short Wheelbase (Dua...,nh,42.834977,-71.107791,2021-04-30T10:15:30-0400,high,2021-04-30 14:15:30+00:00
970,7315044723,https://poconos.craigslist.org/ctd/d/chesterfi...,poconos,https://poconos.craigslist.org,22699,2014.0,jeep,wrangler,gas,110493,clean,NaN,https://images.craigslist.org/00M0M_aFDXWIfug9...,2014 Jeep Wrangler Unlimited Sport 4WD Off...,pa,40.135190,-74.703570,2021-04-30T17:46:04-0400,high,2021-04-30 21:46:04+00:00
1156,7311083008,https://greenville.craigslist.org/ctd/d/greenv...,greenville / upstate,https://greenville.craigslist.org,9998,2013.0,volkswagen,jetta,diesel,96115,clean,NaN,https://images.craigslist.org/00F0F_9oq0KNmZIM...,2013 VOLKSWAGEN JETTA TDI Highline Offered...,sc,34.832550,-82.368350,2021-04-22T17:55:57-0400,medium,2021-04-22 21:55:57+00:00
1349,7303942891,https://wenatchee.craigslist.org/ctd/d/everett...,wenatchee,https://wenatchee.craigslist.org,4995,2005.0,honda,pilot,gas,261591,clean,NaN,https://images.craigslist.org/00S0S_aASnH27HDH...,2005 Honda Pilot EX Offered by: Corn Motor...,wa,47.912956,-122.235725,2021-04-08T16:52:48-0700,low,2021-04-08 23:52:48+00:00
1715,7310344536,https://roanoke.craigslist.org/ctd/d/roanoke-2...,roanoke,https://roanoke.craigslist.org,3995,2007.0,ford,mustang,gas,159342,clean,NaN,https://images.craigslist.org/00J0J_6BuTyEWqNQ...,2007 Ford Mustang V6 Deluxe Coupe Offered ...,va,37.335355,-79.859385,2021-04-21T11:30:46-0400,low,2021-04-21 15:30:46+00:00
1914,7307857568,https://fortmyers.craigslist.org/lee/ctd/d/for...,ft myers / SW florida,https://fortmyers.craigslist.org,17000,2012.0,jeep,grand cherokee,gas,116079,clean,NaN,https://images.craigslist.org/00000_gqxck1dkEu...,2012 Jeep Grand Cherokee Limited Offered b...,fl,26.498960,-81.819380,2021-04-16T14:46:41-0400,medium,2021-04-16 18:46:41+00:00
2023,7314592581,https://pueblo.craigslist.org/ctd/d/pueblo-200...,pueblo,https://pueblo.craigslist.org,8494,2004.0,ford,ranger,gas,156969,clean,NaN,https://images.craigslist.org/01212_6ixSUy4WCq...,2004 Ford Ranger Offered by: Discount Moto...,co,38.298339,-104.616024,2021-04-29T17:30:13-0600,low,2021-04-29 23:30:13+00:00


Заполним незаполненные значения типа топлива значением other

In [26]:
df.transmission = df.transmission.fillna('other')
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,other,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021.0,nissan,altima,gas,8000,clean,automatic,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,ny,40.654800,-73.609700,2021-05-03T18:32:06-0400,medium,2021-05-03 22:32:06+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002.0,toyota,rav4,gas,150000,clean,automatic,https://images.craigslist.org/00n0n_aaEBqmzz5Z...,Transmission:Automatic Exterior Color:WHITE In...,ny,42.123900,-79.189500,2021-04-10T16:33:57-0400,low,2021-04-10 20:33:57+00:00
9996,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008.0,jeep,wrangler,gas,113573,clean,other,https://images.craigslist.org/00000_9ZNVdXaiei...,2008 Jeep Wrangler X Offered by: R&L Auto -- ...,ny,43.216990,-77.755610,2021-05-03T09:36:30-0400,medium,2021-05-03 13:36:30+00:00
9997,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011.0,audi,a3 2.0t premium plus pzev,gas,150184,clean,automatic,https://images.craigslist.org/00X0X_k5JiZDDyqM...,2011 Audi A3 2.0T Premium Plus PZEV Wagon2011 ...,or,44.925908,-122.982753,2021-04-22T12:14:01-0700,low,2021-04-22 19:14:01+00:00
9998,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015.0,porsche,cayenne,hybrid,61943,clean,automatic,https://images.craigslist.org/00O0O_gBihGEQxYT...,"2015 Porsche Cayenne AWD 4dr S E-Hybrid - $31,...",wi,43.029559,-89.397796,2021-04-14T09:14:42-0500,high,2021-04-14 14:14:42+00:00


 Проверяем количество пропущенных значений в типе топлива для обновлённого датафрейма.

In [27]:
quant = df.transmission.isna().sum()
print(f"Количество пропущенных значений в transmission:{quant}")

Количество пропущенных значений в transmission:0


#### Обработка пропусков в долготе и широте(lat, long)

Количество пропущенных значений в долготе и широте

In [28]:
quant_long = df.long.isna().sum()
print(f"Количество пропущенных значений в long:{quant_long}")

Количество пропущенных значений в long:96


In [29]:
 df.long.value_counts(dropna = False)

 NaN           96
-84.112200     88
-84.411811     79
-74.281707     73
-84.445400     54
               ..
-118.194959     1
-93.286303      1
-73.098000      1
-73.732600      1
-79.189500      1
Name: long, Length: 5075, dtype: int64

In [30]:
quant_lat = df.lat.isna().sum()
print(f"Количество пропущенных значений в lat:{quant_lat}")

Количество пропущенных значений в lat:96


Записи, в которых долгота не заполнена

In [31]:
df[df.long.isna()]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
118,7312692841,https://tricities.craigslist.org/ctd/d/bmw-ser...,tri-cities,https://tricities.craigslist.org,17998,2012.0,bmw,5 series,gas,62665,clean,automatic,https://images.craigslist.org/00909_57Xyuj8AJH...,2012 BMW 5 Series 535i M Sport Heat & Cool Sea...,tn,NaN,NaN,2021-04-26T04:28:35-0400,medium,2021-04-26 08:28:35+00:00
126,7315526339,https://newjersey.craigslist.org/ctd/d/look-wh...,north jersey,https://newjersey.craigslist.org,22995,2014.0,bmw,7 series,gas,81085,clean,automatic,https://images.craigslist.org/00k0k_38TNTakyUf...,2014 BMW 7 Series 4dr Sdn 750Li xDrive AWD Aut...,nj,NaN,NaN,2021-05-01T17:49:09-0400,high,2021-05-01 21:49:09+00:00
502,7308641593,https://winstonsalem.craigslist.org/ctd/d/ford...,winston-salem,https://winstonsalem.craigslist.org,20998,2016.0,ford,transit cargo,gas,123825,clean,automatic,https://images.craigslist.org/00101_gwJT4WBNRo...,2016 Ford Transit Cargo Vans Work Van High Roo...,nc,NaN,NaN,2021-04-18T04:43:46-0400,medium,2021-04-18 08:43:46+00:00
567,7309282242,https://onslow.craigslist.org/ctd/d/mercedes-b...,jacksonville,https://onslow.craigslist.org,10810,2014.0,mercedes-benz,c-class,gas,124476,clean,automatic,https://images.craigslist.org/01414_6jNHxQwrKx...,2014 Mercedes Benz C Class C 250 Sport Coupe C...,nc,NaN,NaN,2021-04-19T12:06:28-0400,medium,2021-04-19 16:06:28+00:00
692,7314847134,https://atlanta.craigslist.org/atl/ctd/d/2008-...,atlanta,https://atlanta.craigslist.org,17900,2008.0,toyota,tundra,gas,96000,clean,automatic,https://images.craigslist.org/01313_6u7l4KgvV4...,Description: 2008 Toyota Tundra 5...,ga,NaN,NaN,2021-04-30T12:09:49-0400,medium,2021-04-30 16:09:49+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9154,7312710592,https://athensga.craigslist.org/ctd/d/bmw-x5-a...,athens,https://athensga.craigslist.org,14975,2013.0,bmw,x5,gas,97195,clean,automatic,https://images.craigslist.org/00i0i_kU6D0X9br2...,2013 BMW X5 xDrive35i Sport Activity Navigatio...,ga,NaN,NaN,2021-04-26T07:31:40-0400,medium,2021-04-26 11:31:40+00:00
9408,7311736404,https://raleigh.craigslist.org/ctd/d/lexus-nx-...,raleigh / durham / CH,https://raleigh.craigslist.org,28998,2018.0,lexus,nx 300,gas,19863,clean,automatic,https://images.craigslist.org/00Q0Q_j7S3aRy6Sw...,2018 Lexus NX 300 Navigation Sunroof Bluetooth...,nc,NaN,NaN,2021-04-24T00:54:33-0400,high,2021-04-24 04:54:33+00:00
9650,7315596665,https://newjersey.craigslist.org/ctd/d/wow-201...,north jersey,https://newjersey.craigslist.org,16495,2013.0,jeep,grand cherokee,other,90878,clean,automatic,https://images.craigslist.org/00v0v_awwo3NjD7Q...,2013 Jeep Grand Cherokee 4WD 4dr Laredo Automa...,nj,NaN,NaN,2021-05-01T20:53:57-0400,medium,2021-05-02 00:53:57+00:00
9668,7308677951,https://myrtlebeach.craigslist.org/ctd/d/ford-...,myrtle beach,https://myrtlebeach.craigslist.org,33647,2015.0,ford,f-250 super duty,diesel,153524,clean,automatic,https://images.craigslist.org/00f0f_1c0aGXmmCL...,2015 Ford F250 4x4 Truck Diesel Pickup Trucks ...,sc,NaN,NaN,2021-04-18T08:43:25-0400,high,2021-04-18 12:43:25+00:00


Штаты, в которых находятся автомобили с незаполненной долготой

In [32]:
df[df.long.isna()].state.value_counts()

nc    25
va    14
ga    10
ct     9
fl     6
tn     5
nj     4
ny     4
ok     3
sc     3
oh     2
mo     2
ma     2
co     2
id     1
ia     1
wa     1
tx     1
ar     1
Name: state, dtype: int64

Загрузим файл с названиями штатов, их долготой и широтой (взят с сайта https://developers.google.com/)

In [34]:
df_states=pd.read_csv('data/states.csv')
df_states.head()

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


Преобразовываем загруженный датафрейм в словарь

In [35]:
dict_frame=df_states.to_dict()
dict_frame

{'state': {0: 'AK',
  1: 'AL',
  2: 'AR',
  3: 'AZ',
  4: 'CA',
  5: 'CO',
  6: 'CT',
  7: 'DC',
  8: 'DE',
  9: 'FL',
  10: 'GA',
  11: 'HI',
  12: 'IA',
  13: 'ID',
  14: 'IL',
  15: 'IN',
  16: 'KS',
  17: 'KY',
  18: 'LA',
  19: 'MA',
  20: 'MD',
  21: 'ME',
  22: 'MI',
  23: 'MN',
  24: 'MO',
  25: 'MS',
  26: 'MT',
  27: 'NC',
  28: 'ND',
  29: 'NE',
  30: 'NH',
  31: 'NJ',
  32: 'NM',
  33: 'NV',
  34: 'NY',
  35: 'OH',
  36: 'OK',
  37: 'OR',
  38: 'PA',
  39: 'PR',
  40: 'RI',
  41: 'SC',
  42: 'SD',
  43: 'TN',
  44: 'TX',
  45: 'UT',
  46: 'VA',
  47: 'VT',
  48: 'WA',
  49: 'WI',
  50: 'WV',
  51: 'WY'},
 'latitude': {0: 63.588753,
  1: 32.318231,
  2: 35.20105,
  3: 34.048928,
  4: 36.778261,
  5: 39.550051,
  6: 41.603221,
  7: 38.905985,
  8: 38.910832,
  9: 27.664827,
  10: 32.157435,
  11: 19.898682,
  12: 41.878003,
  13: 44.068202,
  14: 40.633125,
  15: 40.551217,
  16: 39.011902,
  17: 37.839333,
  18: 31.244823,
  19: 42.407211,
  20: 39.045755,
  21: 45.253783,
 

Создаём список штатов

In [36]:
list_states=[]
for val in dict_frame['state'].values():
    list_states.append(val.lower())
list_states

['ak',
 'al',
 'ar',
 'az',
 'ca',
 'co',
 'ct',
 'dc',
 'de',
 'fl',
 'ga',
 'hi',
 'ia',
 'id',
 'il',
 'in',
 'ks',
 'ky',
 'la',
 'ma',
 'md',
 'me',
 'mi',
 'mn',
 'mo',
 'ms',
 'mt',
 'nc',
 'nd',
 'ne',
 'nh',
 'nj',
 'nm',
 'nv',
 'ny',
 'oh',
 'ok',
 'or',
 'pa',
 'pr',
 'ri',
 'sc',
 'sd',
 'tn',
 'tx',
 'ut',
 'va',
 'vt',
 'wa',
 'wi',
 'wv',
 'wy']

Создаём список долготы

In [37]:
list_longs=[]
for val in dict_frame['longitude'].values():
    list_longs.append(val)
list_longs

[-154.493062,
 -86.902298,
 -91.831833,
 -111.093731,
 -119.417932,
 -105.782067,
 -73.087749,
 -77.033418,
 -75.52767,
 -81.515754,
 -82.907123,
 -155.665857,
 -93.097702,
 -114.742041,
 -89.398528,
 -85.602364,
 -98.484246,
 -84.270018,
 -92.145024,
 -71.382437,
 -76.641271,
 -69.445469,
 -85.602364,
 -94.6859,
 -91.831833,
 -89.398528,
 -110.362566,
 -79.0193,
 -101.002012,
 -99.901813,
 -71.572395,
 -74.405661,
 -105.032363,
 -116.419389,
 -74.217933,
 -82.907123,
 -97.092877,
 -120.554201,
 -77.194525,
 -66.590149,
 -71.477429,
 -81.163725,
 -99.901813,
 -86.580447,
 -99.901813,
 -111.093731,
 -78.656894,
 -72.577841,
 -120.740139,
 -88.787868,
 -80.454903,
 -107.290284]

Создаём список широты

In [38]:
list_lats=[]
for val in dict_frame['latitude'].values():
    list_lats.append(val)
list_lats

[63.588753,
 32.318231,
 35.20105,
 34.048928,
 36.778261,
 39.550051,
 41.603221,
 38.905985,
 38.910832,
 27.664827,
 32.157435,
 19.898682,
 41.878003,
 44.068202,
 40.633125,
 40.551217,
 39.011902,
 37.839333,
 31.244823,
 42.407211,
 39.045755,
 45.253783,
 44.314844,
 46.729553,
 37.964253,
 32.354668,
 46.879682,
 35.759573,
 47.551493,
 41.492537,
 43.193852,
 40.058324,
 34.97273,
 38.80261,
 43.299428,
 40.417287,
 35.007752,
 43.804133,
 41.203322,
 18.220833,
 41.580095,
 33.836081,
 43.969515,
 35.517491,
 31.968599,
 39.32098,
 37.431573,
 44.558803,
 47.751074,
 43.78444,
 38.597626,
 43.075968]

Создаем словарь, где ключами являются наименования штатов, а значениями - долгота соответствующего штата

In [39]:
dictionary_long = dict(zip(list_states, list_longs))
print(dictionary_long)

{'ak': -154.493062, 'al': -86.902298, 'ar': -91.831833, 'az': -111.093731, 'ca': -119.417932, 'co': -105.782067, 'ct': -73.087749, 'dc': -77.033418, 'de': -75.52767, 'fl': -81.515754, 'ga': -82.907123, 'hi': -155.665857, 'ia': -93.097702, 'id': -114.742041, 'il': -89.398528, 'in': -85.602364, 'ks': -98.484246, 'ky': -84.270018, 'la': -92.145024, 'ma': -71.382437, 'md': -76.641271, 'me': -69.445469, 'mi': -85.602364, 'mn': -94.6859, 'mo': -91.831833, 'ms': -89.398528, 'mt': -110.362566, 'nc': -79.0193, 'nd': -101.002012, 'ne': -99.901813, 'nh': -71.572395, 'nj': -74.405661, 'nm': -105.032363, 'nv': -116.419389, 'ny': -74.217933, 'oh': -82.907123, 'ok': -97.092877, 'or': -120.554201, 'pa': -77.194525, 'pr': -66.590149, 'ri': -71.477429, 'sc': -81.163725, 'sd': -99.901813, 'tn': -86.580447, 'tx': -99.901813, 'ut': -111.093731, 'va': -78.656894, 'vt': -72.577841, 'wa': -120.740139, 'wi': -88.787868, 'wv': -80.454903, 'wy': -107.290284}


Создаем словарь, где ключами являются наименования штатов, а значениями - широта соответствующего штата

In [40]:
dictionary_lat = dict(zip(list_states, list_lats))
print(dictionary_lat)

{'ak': 63.588753, 'al': 32.318231, 'ar': 35.20105, 'az': 34.048928, 'ca': 36.778261, 'co': 39.550051, 'ct': 41.603221, 'dc': 38.905985, 'de': 38.910832, 'fl': 27.664827, 'ga': 32.157435, 'hi': 19.898682, 'ia': 41.878003, 'id': 44.068202, 'il': 40.633125, 'in': 40.551217, 'ks': 39.011902, 'ky': 37.839333, 'la': 31.244823, 'ma': 42.407211, 'md': 39.045755, 'me': 45.253783, 'mi': 44.314844, 'mn': 46.729553, 'mo': 37.964253, 'ms': 32.354668, 'mt': 46.879682, 'nc': 35.759573, 'nd': 47.551493, 'ne': 41.492537, 'nh': 43.193852, 'nj': 40.058324, 'nm': 34.97273, 'nv': 38.80261, 'ny': 43.299428, 'oh': 40.417287, 'ok': 35.007752, 'or': 43.804133, 'pa': 41.203322, 'pr': 18.220833, 'ri': 41.580095, 'sc': 33.836081, 'sd': 43.969515, 'tn': 35.517491, 'tx': 31.968599, 'ut': 39.32098, 'va': 37.431573, 'vt': 44.558803, 'wa': 47.751074, 'wi': 43.78444, 'wv': 38.597626, 'wy': 43.075968}


Создаём маску замены и, зная название штата, заменим пропущенные значения долготы и широты на значения из states.csv

In [41]:
mask_long = df['long'].astype('str').isin(['NaN','nan'])
df.loc[mask_long, 'long'] = df.loc[mask_long, 'state'].map(dictionary_long)
mask_lat= df['lat'].astype('str').isin(['NaN','nan'])
df.loc[mask_lat, 'lat'] = df.loc[mask_lat, 'state'].map(dictionary_long)
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,other,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021.0,nissan,altima,gas,8000,clean,automatic,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,ny,40.654800,-73.609700,2021-05-03T18:32:06-0400,medium,2021-05-03 22:32:06+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002.0,toyota,rav4,gas,150000,clean,automatic,https://images.craigslist.org/00n0n_aaEBqmzz5Z...,Transmission:Automatic Exterior Color:WHITE In...,ny,42.123900,-79.189500,2021-04-10T16:33:57-0400,low,2021-04-10 20:33:57+00:00
9996,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008.0,jeep,wrangler,gas,113573,clean,other,https://images.craigslist.org/00000_9ZNVdXaiei...,2008 Jeep Wrangler X Offered by: R&L Auto -- ...,ny,43.216990,-77.755610,2021-05-03T09:36:30-0400,medium,2021-05-03 13:36:30+00:00
9997,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011.0,audi,a3 2.0t premium plus pzev,gas,150184,clean,automatic,https://images.craigslist.org/00X0X_k5JiZDDyqM...,2011 Audi A3 2.0T Premium Plus PZEV Wagon2011 ...,or,44.925908,-122.982753,2021-04-22T12:14:01-0700,low,2021-04-22 19:14:01+00:00
9998,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015.0,porsche,cayenne,hybrid,61943,clean,automatic,https://images.craigslist.org/00O0O_gBihGEQxYT...,"2015 Porsche Cayenne AWD 4dr S E-Hybrid - $31,...",wi,43.029559,-89.397796,2021-04-14T09:14:42-0500,high,2021-04-14 14:14:42+00:00


 Проверяем количество пропущенных значений в широте и долготе выпуска автомобиля для обновлённого датафрейма.

In [42]:
quant_long = df.long.isna().sum()
print(f"Количество пропущенных значений в long:{quant_long}")
quant_lat = df.lat.isna().sum()
print(f"Количество пропущенных значений в lat:{quant_lat}")

Количество пропущенных значений в long:0
Количество пропущенных значений в lat:0


#### Проверка пропусков

Количество пропущенных значений в дате

In [43]:
missing_values_out_clean = ((df.isna().sum() / len(df))*100).sort_values()[::-1]
print('Процент пропущенных значений: ')
missing_values_out_clean

Процент пропущенных значений: 


date              0.0
price_category    0.0
url               0.0
region            0.0
region_url        0.0
price             0.0
year              0.0
manufacturer      0.0
model             0.0
fuel              0.0
odometer          0.0
title_status      0.0
transmission      0.0
image_url         0.0
description       0.0
state             0.0
lat               0.0
long              0.0
posting_date      0.0
id                0.0
dtype: float64

### Преобразование данных

 Выведем информацию о датафрейме, чтобы изучить типы данных

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9964 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9964 non-null   int64  
 1   url             9964 non-null   object 
 2   region          9964 non-null   object 
 3   region_url      9964 non-null   object 
 4   price           9964 non-null   int64  
 5   year            9964 non-null   float64
 6   manufacturer    9964 non-null   object 
 7   model           9964 non-null   object 
 8   fuel            9964 non-null   object 
 9   odometer        9964 non-null   int64  
 10  title_status    9964 non-null   object 
 11  transmission    9964 non-null   object 
 12  image_url       9964 non-null   object 
 13  description     9964 non-null   object 
 14  state           9964 non-null   object 
 15  lat             9964 non-null   float64
 16  long            9964 non-null   float64
 17  posting_date    9964 non-null   o

#### Преобразование типа данных года автомобиля

Выводим описательные статистики для года выпуска автомобиля

In [45]:
df.year.describe(include = 'all')

count    9964.000000
mean     2010.927941
std         9.672040
min      1915.000000
25%      2008.000000
50%      2013.000000
75%      2017.000000
max      2022.000000
Name: year, dtype: float64

Изменим тип данных для года выпуска на более подходящий целочисленный тип

In [46]:
df.year = df.year.astype(int)
df.year.dtype

dtype('int32')

#### Преобразование данных о виде топлива в категориальные переменные

In [47]:
df.fuel = df.fuel.astype(str)
df.fuel.dtype

dtype('O')

In [48]:
series_fuel = pd.Series(df.fuel, dtype="category")
series_fuel

0       diesel
1        other
2          gas
3          gas
4          gas
         ...  
9995       gas
9996       gas
9997       gas
9998    hybrid
9999       gas
Name: fuel, Length: 9964, dtype: category
Categories (5, object): ['diesel', 'electric', 'gas', 'hybrid', 'other']

#### Преобразование данных о коробке передач в категориальные переменные

In [49]:
df.transmission = df.transmission.astype(str)
df.transmission.dtype

dtype('O')

In [50]:
series_state = pd.Series(df.transmission, dtype="category")
series_state

0           other
1       automatic
2           other
3       automatic
4       automatic
          ...    
9995    automatic
9996        other
9997    automatic
9998    automatic
9999        other
Name: transmission, Length: 9964, dtype: category
Categories (3, object): ['automatic', 'manual', 'other']

#### Преобразование данных о регионе в категориальные переменные

In [51]:
df.region = df.region.astype(str)
df.region.dtype

dtype('O')

In [52]:
series_state = pd.Series(df.region, dtype="category")
series_state

0                   chattanooga
1                  north jersey
2                  reno / tahoe
3                  fayetteville
4                 new york city
                 ...           
9995                 chautauqua
9996                 binghamton
9997                      salem
9998                    madison
9999    norfolk / hampton roads
Name: region, Length: 9964, dtype: category
Categories (393, object): ['SF bay area', 'abilene', 'akron / canton', 'albany', ..., 'youngstown', 'yuba-sutter', 'yuma', 'zanesville / cambridge']

#### Преобразование данных о производителе в категориальные переменные

In [53]:
df.manufacturer = df.manufacturer.astype(str)
df.manufacturer.dtype

dtype('O')

In [54]:
series_state = pd.Series(df.manufacturer, dtype="category")
series_state

0              ram
1             ford
2       volkswagen
3           toyota
4           nissan
           ...    
9995        toyota
9996          jeep
9997          audi
9998       porsche
9999           ram
Name: manufacturer, Length: 9964, dtype: category
Categories (40, object): ['acura', 'alfa-romeo', 'audi', 'bmw', ..., 'tesla', 'toyota', 'volkswagen', 'volvo']

#### Преобразование данных о штате в категориальные переменные

In [55]:
df.state = df.state.astype(str)
df.state.dtype

dtype('O')

In [56]:
series_state = pd.Series(df.state, dtype="category")
series_state

0       tn
1       nj
2       ca
3       nc
4       ny
        ..
9995    ny
9996    ny
9997    or
9998    wi
9999    va
Name: state, Length: 9964, dtype: category
Categories (51, object): ['ak', 'al', 'ar', 'az', ..., 'wa', 'wi', 'wv', 'wy']

#### Преобразование данных о статусе в категориальные переменные

In [57]:
df.title_status = df.title_status.astype(str)
df.title_status.dtype

dtype('O')

In [58]:
series_state = pd.Series(df.title_status, dtype="category")
series_state

0       clean
1       clean
2       clean
3       clean
4       clean
        ...  
9995    clean
9996    clean
9997    clean
9998    clean
9999    clean
Name: title_status, Length: 9964, dtype: category
Categories (7, object): ['<bound method Series.mode of 0       clean\n1..., 'clean', 'lien', 'missing', 'parts only', 'rebuilt', 'salvage']

### Выбросы в данных

#### Выбросы в годе выпуска

Описательные статистики для года выпуска автомобиля.


In [59]:
df.year.describe(include = 'all')

count    9964.000000
mean     2010.927941
std         9.672040
min      1915.000000
25%      2008.000000
50%      2013.000000
75%      2017.000000
max      2022.000000
Name: year, dtype: float64

Посмотрим границы для нормальных значений года выпуска, используя интерквартильный размах.


In [60]:
def calculate_outliers(data):
    q25 = data.quantile(0.25)
    q75 = data.quantile(0.75)
    iqr = q75 - q25
    boundaries = (q25 - 1.5 * iqr, q75 + 1.5 * iqr)

    return boundaries
boundaries = calculate_outliers(df.year)
boundaries

(1994.5, 2030.5)

Посчитаем количество выбросов в годе выпуска автомобиля.


In [61]:
is_outlier = (df.year < boundaries[0]) | (df.year > boundaries[1])
is_outlier.sum()

394

Посчитаем долю выбросов в годе выпуска автомобиля.


In [62]:
is_outlier.sum() / len(df)

0.039542352468887994

5. Выведите записи, в которых год выпуска попадает в выбросы. Посмотрите на данные и подумайте, могут ли эти выбросы описывать особые значения категории цены или других признаков или действительно являются выбросами.


In [63]:
df[is_outlier]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
13,7317089086,https://seattle.craigslist.org/see/ctd/d/portl...,seattle-tacoma,https://seattle.craigslist.org,14995,1994,ford,e-series van universal by glaval designer luxu...,gas,156925,clean,automatic,https://images.craigslist.org/00r0r_13JUlXZOMl...,"Priced to Sell: $14,9951994 Ford E-Series Van ...",wa,45.391373,-122.613533,2021-05-04T20:22:11-0700,medium,2021-05-05 03:22:11+00:00
21,7312327909,https://madison.craigslist.org/cto/d/madison-1...,madison,https://madison.craigslist.org,89995,1970,other,oldsmobile 442,gas,111111,clean,automatic,https://images.craigslist.org/00S0S_agk7Nol4CO...,1970 Oldsmobile 442 Convertible 442 Indy Pace ...,wi,43.098400,-89.273400,2021-04-25T10:10:43-0500,high,2021-04-25 15:10:43+00:00
57,7314413105,https://inlandempire.craigslist.org/ctd/d/cost...,inland empire,https://inlandempire.craigslist.org,69888,1948,ford,coe,gas,1,clean,automatic,https://images.craigslist.org/00M0M_8jgBCZZuG6...,Awesome Resto-Mod COE Ford Car Hauler. Equippe...,ca,33.680100,-117.908500,2021-04-29T10:49:29-0700,high,2021-04-29 17:49:29+00:00
66,7310077530,https://hudsonvalley.craigslist.org/cto/d/bloo...,hudson valley,https://hudsonvalley.craigslist.org,5500,1990,other,Diahatsu,gas,30000,clean,manual,https://images.craigslist.org/00T0T_a4VRbx40ea...,1991 Daihatsu hijet 5 speed extended cab....im...,ny,41.564400,-74.430400,2021-04-20T18:30:18-0400,low,2021-04-20 22:30:18+00:00
106,7314527175,https://rapidcity.craigslist.org/ctd/d/sherman...,rapid city / west SD,https://rapidcity.craigslist.org,43500,1968,chevrolet,camaro,gas,43320,clean,manual,https://images.craigslist.org/00h0h_2N66LMJBAI...,1968 Chevrolet Camaro Convertible. VIN:124678N...,sd,33.637200,-96.618400,2021-04-29T15:11:01-0600,high,2021-04-29 21:11:01+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9886,7313487583,https://springfield.craigslist.org/cto/d/mount...,springfield,https://springfield.craigslist.org,3500,1974,jeep,cj5,gas,10000,clean,manual,https://images.craigslist.org/00G0G_cdxzaTaVqa...,1974 Jeep cj5 304 v8 4speed manual trans Da...,mo,36.331200,-92.375300,2021-04-27T14:30:28-0500,low,2021-04-27 19:30:28+00:00
9921,7311566144,https://chico.craigslist.org/cto/d/chico-1994-...,chico,https://chico.craigslist.org,16000,1994,ford,f250,gas,32500,clean,automatic,https://images.craigslist.org/01010_343KkgTaHo...,"Up for sale is this 32,000 original mile Ford ...",ca,39.745800,-121.844400,2021-04-23T13:51:20-0700,medium,2021-04-23 20:51:20+00:00
9944,7313676859,https://reno.craigslist.org/ctd/d/sherman-1984...,reno / tahoe,https://reno.craigslist.org,26500,1984,chevrolet,silverado,gas,38751,clean,automatic,https://images.craigslist.org/00r0r_czXfi4UeJF...,"1984 Chevrolet C10 Silverado, short wheelbase ...",ca,33.637200,-96.618400,2021-04-27T19:44:27-0700,high,2021-04-28 02:44:27+00:00
9972,7316458200,https://portland.craigslist.org/mlt/cto/d/port...,portland,https://portland.craigslist.org,18500,1972,chevrolet,other,gas,75000,clean,automatic,https://images.craigslist.org/00x0x_7FqS8iHJst...,1972 classic Chevrolet pick up custom deluxe ...,or,45.494960,-122.666988,2021-05-03T14:58:34-0700,medium,2021-05-03 21:58:34+00:00


Количество значений для категории цены для записей, в которых год выпуска автомобиля является выбросом.

In [64]:
df[is_outlier].price_category.value_counts(dropna = False)

low       181
medium    135
high       78
Name: price_category, dtype: int64

Так как выбросы в годе выпуска находятся среди минимальных значений, заполним те значения, которые являются выбросами, значением нижней границы.


In [65]:
df.loc[is_outlier, 'year'] = int(boundaries[0])

Выведем ещё раз описательные статистики для года выпуска автомобиля, чтобы проверить, что теперь все значения года входят в границы нормальных значений.

In [66]:
df.year.describe(include = 'all')

count    9964.000000
mean     2011.669410
std         6.516981
min      1994.000000
25%      2008.000000
50%      2013.000000
75%      2017.000000
max      2022.000000
Name: year, dtype: float64

# Feature Engineering

## Построение признаков из строк

Выведем список значений и количество уникальных значений для модели автомобиля.

In [67]:
df["model"].unique()


array(['2500 crew cab big horn', 'explorer 4wd 4dr xlt',
       'golf r hatchback', ..., 'navigator reserve', '1988 Nisan',
       'a3 2.0t premium plus pzev'], dtype=object)

In [68]:
df["model"].nunique()

3447

Количество уникальных значений велико. Попробуем их сократить, оставив только первое слово в наименовании модели.

In [69]:
df['short_model'] = df.model.apply(lambda x: x.split(' ')[0])
df['short_model']

0           2500
1       explorer
2           golf
3           rav4
4         altima
          ...   
9995        rav4
9996    wrangler
9997          a3
9998     cayenne
9999        1500
Name: short_model, Length: 9964, dtype: object

In [70]:
df['short_model'].nunique()

973

Преобразуем данные 'short_model' в категориальные переменные

In [71]:
df.short_model = df.short_model.astype(str)
df.short_model.dtype

dtype('O')

In [72]:
series_state = pd.Series(df.short_model, dtype="category")
series_state

0           2500
1       explorer
2           golf
3           rav4
4         altima
          ...   
9995        rav4
9996    wrangler
9997          a3
9998     cayenne
9999        1500
Name: short_model, Length: 9964, dtype: category
Categories (973, object): ['-benz', '1', '124', '128i', ..., 'z4', 'zephyr', 'zx2', '♿']

## Преобразование категориальных переменных

 Для категориальных переменных `short_model`, `transmission`, `region`, `fuel`,`manufacturer`, `state`, `title_status` распечатаем наименование категории и количество уникальных значений в ней.


In [73]:
for cat_var in ["short_model", "transmission", "region", 'manufacturer', 'state', 'title_status', 'fuel']:
    print ('Количество уникальных значений в категории "',cat_var,'": ', df[cat_var].nunique())
    print ('---------------------------------------------------')

Количество уникальных значений в категории " short_model ":  973
---------------------------------------------------
Количество уникальных значений в категории " transmission ":  3
---------------------------------------------------
Количество уникальных значений в категории " region ":  393
---------------------------------------------------
Количество уникальных значений в категории " manufacturer ":  40
---------------------------------------------------
Количество уникальных значений в категории " state ":  51
---------------------------------------------------
Количество уникальных значений в категории " title_status ":  7
---------------------------------------------------
Количество уникальных значений в категории " fuel ":  5
---------------------------------------------------


Создадим датафрейм `data`, содержащий только выбранные категориальные переменные.

In [74]:
data = df[["short_model", "transmission", "region", 'manufacturer', 'state', 'title_status', 'fuel']]
data

,short_model,transmission,region,manufacturer,state,title_status,fuel
0,2500,other,chattanooga,ram,tn,clean,diesel
1,explorer,automatic,north jersey,ford,nj,clean,other
2,golf,other,reno / tahoe,volkswagen,ca,clean,gas
3,rav4,automatic,fayetteville,toyota,nc,clean,gas
4,altima,automatic,new york city,nissan,ny,clean,gas
...,...,...,...,...,...,...,...
9995,rav4,automatic,chautauqua,toyota,ny,clean,gas
9996,wrangler,other,binghamton,jeep,ny,clean,gas
9997,a3,automatic,salem,audi,or,clean,gas
9998,cayenne,automatic,madison,porsche,wi,clean,hybrid


Создадим объект класса OneHotEncoder, который будет отвечать за преобразование.

In [75]:
ohe = OneHotEncoder(sparse = False)
ohe

OneHotEncoder(sparse=False)

Идентифицируем параметры преобразования на датафрейме с категориальными переменными.

In [76]:
ohe.fit(data)

OneHotEncoder(sparse=False)

In [77]:
ohe_data = ohe.transform(data)
ohe_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

Применим преобразование для датафрейма с категориальными переменными и сохраним в новую переменную, распечатаем получившуюся матрицу и её размерность.

In [78]:
ohe_data = ohe.transform(data)
ohe_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [79]:
ohe_data.shape

(9964, 1472)

Выведем новые наименования признаков с помощью метода `get_feature_names_out`.

In [80]:
ohe.get_feature_names_out()

array(['short_model_-benz', 'short_model_1', 'short_model_124', ...,
       'fuel_gas', 'fuel_hybrid', 'fuel_other'], dtype=object)

Добавим в исходный датафрейм получившиеся новые признаки, выведем первые пять записей датафрейма и список колонок.

In [81]:
df[ohe.get_feature_names_out()] = ohe_data

C:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [82]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021,nissan,altima,gas,8000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [83]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer',
       ...
       'title_status_lien', 'title_status_missing', 'title_status_parts only',
       'title_status_rebuilt', 'title_status_salvage', 'fuel_diesel',
       'fuel_electric', 'fuel_gas', 'fuel_hybrid', 'fuel_other'],
      dtype='object', length=1493)

Посмотрим на результат обратного преобразования получившейся матрицы в исходные признаки, используем `inverse_transform`.

In [84]:
ohe.inverse_transform(ohe_data)

array([['2500', 'other', 'chattanooga', ..., 'tn', 'clean', 'diesel'],
       ['explorer', 'automatic', 'north jersey', ..., 'nj', 'clean',
        'other'],
       ['golf', 'other', 'reno / tahoe', ..., 'ca', 'clean', 'gas'],
       ...,
       ['a3', 'automatic', 'salem', ..., 'or', 'clean', 'gas'],
       ['cayenne', 'automatic', 'madison', ..., 'wi', 'clean', 'hybrid'],
       ['1500', 'other', 'norfolk / hampton roads', ..., 'va', 'clean',
        'gas']], dtype=object)

## Стандартизация данных

Создадим переменную под количественные переменные `lat`, `long`, `year`, `odometer/price`, `desc_len`, `model_in_desc`, `model_len`, `model_word_count` и создайте датафрейм `data`, содержащий только выбранные количественные переменные.


In [85]:
quan_var = ["lat", "long", "year", 'odometer', 'price']
data = df[quan_var]
data

,lat,long,year,odometer,price
0,35.060000,-85.250000,2020,27442,54990
1,40.821805,-74.061962,2016,60023,16942
2,33.779214,-84.411811,2017,14048,35590
3,35.715954,-78.655304,2013,117291,14500
4,40.654800,-73.609700,2021,8000,21800
...,...,...,...,...,...
9995,42.123900,-79.189500,2002,150000,4495
9996,43.216990,-77.755610,2008,113573,14495
9997,44.925908,-122.982753,2011,150184,8995
9998,43.029559,-89.397796,2015,61943,31900


Создадим объект `std_scaler` класса StandardScaler с параметрами по умолчанию, который будет отвечать за стандартизацию.

In [86]:
std_scaler = StandardScaler()
std_scaler

StandardScaler()

Идентифицируем параметры преобразования на датафрейме с количественными переменными.

In [87]:
std_scaler.fit(data)

StandardScaler()

Применим преобразование для датафрейма с количественными переменными и сохраним в новую переменную, распечатаем получившуюся матрицу и её размерность.

In [88]:
std_scaled = std_scaler.transform(data)
std_scaled

array([[-0.18150167,  0.4842838 ,  1.27835384, -1.04790281,  0.27550068],
       [ 0.2571456 ,  1.1022232 ,  0.66454194, -0.5317451 , -0.02930064],
       [-0.27900814,  0.53057878,  0.81799491, -1.26009446,  0.1200879 ],
       ...,
       [ 0.5695917 , -1.59977731, -0.10272294,  0.89661177, -0.0929638 ],
       [ 0.42522231,  0.2551921 ,  0.51108896, -0.5013279 ,  0.09052742],
       [-0.05893207,  0.97861155,  0.81799491, -0.91357602,  0.09124841]])

Создадим наименования новых стандартизированных признаков.

In [89]:
df[["lat_std", "long_std", "year_std", 'odometer_std', 'price_std']] = std_scaled

Добавим в исходный датафрейм получившиеся новые признаки, выведем первые пять записей датафрейма и список колонок.

In [90]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,lat_std,long_std,year_std,odometer_std,price_std
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,1.0,0.0,0.0,0.0,0.0,-0.181502,0.484284,1.278354,-1.047903,0.275501
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.0,0.0,0.0,0.0,1.0,0.257146,1.102223,0.664542,-0.531745,-0.029301
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.0,0.0,1.0,0.0,0.0,-0.279008,0.530579,0.817995,-1.260094,0.120088
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.0,0.0,1.0,0.0,0.0,-0.131564,0.848523,0.204183,0.375511,-0.048863
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021,nissan,altima,gas,8000,...,0.0,0.0,1.0,0.0,0.0,0.244431,1.127203,1.431807,-1.355909,0.009617


In [91]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer',
       ...
       'fuel_diesel', 'fuel_electric', 'fuel_gas', 'fuel_hybrid', 'fuel_other',
       'lat_std', 'long_std', 'year_std', 'odometer_std', 'price_std'],
      dtype='object', length=1498)

## Признаки на основе дат

Выведем по отдельности колонки `posting_date` и `date`.

In [92]:
df['posting_date']

0       2021-04-17T12:30:50-0400
1       2021-05-03T15:40:21-0400
2       2021-04-28T03:52:20-0700
3       2021-04-17T10:08:57-0400
4       2021-05-03T18:32:06-0400
                  ...           
9995    2021-04-10T16:33:57-0400
9996    2021-05-03T09:36:30-0400
9997    2021-04-22T12:14:01-0700
9998    2021-04-14T09:14:42-0500
9999    2021-04-24T13:50:49-0400
Name: posting_date, Length: 9964, dtype: object

In [93]:
df['date']

0       2021-04-17 16:30:50+00:00
1       2021-05-03 19:40:21+00:00
2       2021-04-28 10:52:20+00:00
3       2021-04-17 14:08:57+00:00
4       2021-05-03 22:32:06+00:00
                  ...            
9995    2021-04-10 20:33:57+00:00
9996    2021-05-03 13:36:30+00:00
9997    2021-04-22 19:14:01+00:00
9998    2021-04-14 14:14:42+00:00
9999    2021-04-24 17:50:49+00:00
Name: date, Length: 9964, dtype: object

Изменим тип `date`c помощью `pd.to_datetime`.

In [94]:
df['date'] = pd.to_datetime(df['date'])
df['date']

0      2021-04-17 16:30:50+00:00
1      2021-05-03 19:40:21+00:00
2      2021-04-28 10:52:20+00:00
3      2021-04-17 14:08:57+00:00
4      2021-05-03 22:32:06+00:00
                  ...           
9995   2021-04-10 20:33:57+00:00
9996   2021-05-03 13:36:30+00:00
9997   2021-04-22 19:14:01+00:00
9998   2021-04-14 14:14:42+00:00
9999   2021-04-24 17:50:49+00:00
Name: date, Length: 9964, dtype: datetime64[ns, UTC]

Создадим признак `month`, обозначающий номер месяца размещения объявления о продаже автомобиля, и распечатаем его.

In [95]:
df['month'] = df.date.dt.month
df['month']

0       4
1       5
2       4
3       4
4       5
       ..
9995    4
9996    5
9997    4
9998    4
9999    4
Name: month, Length: 9964, dtype: int64

Создадим признак `dayofweek`, обозначающий день недели размещения объявления о продаже автомобиля, и распечатаем его.

In [96]:
df['dayofweek'] = df.date.dt.dayofweek
df['dayofweek']

0       5
1       0
2       2
3       5
4       0
       ..
9995    5
9996    0
9997    3
9998    2
9999    5
Name: dayofweek, Length: 9964, dtype: int64

Создадим признак `diff_years`, обозначающий количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля, и распечатаем его.

In [97]:
df['diff_years'] = df.apply(lambda x: x.date.year - x.year, axis = 1)
df['diff_years']

0        1
1        5
2        4
3        8
4        0
        ..
9995    19
9996    13
9997    10
9998     6
9999     4
Name: diff_years, Length: 9964, dtype: int64

Применим стандартизацию к новым признаками и сохраним результат в новые признаки вида `month_std`.

In [98]:
std_scaler = StandardScaler()

In [99]:
data = df[['month','dayofweek','diff_years']]
data

,month,dayofweek,diff_years
0,4,5,1
1,5,0,5
2,4,2,4
3,4,5,8
4,5,0,0
...,...,...,...
9995,4,5,19
9996,5,0,13
9997,4,3,10
9998,4,2,6


In [100]:
std_scaler.fit(data)


StandardScaler()

In [101]:
std_scaled = std_scaler.transform(data)
std_scaled

array([[-0.61230201,  1.11444276, -1.27835384],
       [ 1.63318097, -1.37441945, -0.66454194],
       [-0.61230201, -0.37887457, -0.81799491],
       ...,
       [-0.61230201,  0.11889787,  0.10272294],
       [-0.61230201, -0.37887457, -0.51108896],
       [-0.61230201,  1.11444276, -0.81799491]])

In [102]:
df[["month_std", "dayofweek_std", "diff_years_std"]] = std_scaled
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,long_std,year_std,odometer_std,price_std,month,dayofweek,diff_years,month_std,dayofweek_std,diff_years_std
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0.484284,1.278354,-1.047903,0.275501,4,5,1,-0.612302,1.114443,-1.278354
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,1.102223,0.664542,-0.531745,-0.029301,5,0,5,1.633181,-1.374419,-0.664542
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.530579,0.817995,-1.260094,0.120088,4,2,4,-0.612302,-0.378875,-0.817995
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.848523,0.204183,0.375511,-0.048863,4,5,8,-0.612302,1.114443,-0.204183
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021,nissan,altima,gas,8000,...,1.127203,1.431807,-1.355909,0.009617,5,0,0,1.633181,-1.374419,-1.431807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002,toyota,rav4,gas,150000,...,0.819018,-1.483800,0.893697,-0.129013,4,5,19,-0.612302,1.114443,1.483800
9996,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008,jeep,wrangler,gas,113573,...,0.898215,-0.563082,0.316610,-0.048903,5,0,13,1.633181,-1.374419,0.563082
9997,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011,audi,a3 2.0t premium plus pzev,gas,150184,...,-1.599777,-0.102723,0.896612,-0.092964,4,3,10,-0.612302,0.118898,0.102723
9998,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015,porsche,cayenne,hybrid,61943,...,0.255192,0.511089,-0.501328,0.090527,4,2,6,-0.612302,-0.378875,-0.511089


## Сохранение датафрейма для этапа моделирования

Удалим колонки, которые были исходными для формирования признаков, а также новые признаки, которые были добавлены с целью обучения.  Сохраните новый датафрейм в переменную `df_prepared`.

In [103]:
columns_for_drop = ['year', 'url', 'region', 'region_url', 'manufacturer',
                    'model', 'fuel', 'odometer', 'title_status', 'transmission',
                    'image_url', 'description', 'state', 'lat', 'long', 'posting_date',
                    'short_model', 'date', 'month', 'dayofweek', 'diff_years']
df_prepared = df.drop(columns = columns_for_drop,axis = 1)

2. Распечатайте оставшийся список колонок и размерность финального датафрейма.

In [104]:
df_prepared.columns.values.tolist()

['id',
 'price',
 'price_category',
 'short_model_-benz',
 'short_model_1',
 'short_model_124',
 'short_model_128i',
 'short_model_135i',
 'short_model_1500',
 'short_model_163869',
 'short_model_1928',
 'short_model_1951',
 'short_model_1954',
 'short_model_1960s',
 'short_model_1966',
 'short_model_1968',
 'short_model_1972',
 'short_model_1977',
 'short_model_1978',
 'short_model_198.5',
 'short_model_1988',
 'short_model_1994',
 'short_model_1999',
 'short_model_2',
 'short_model_200',
 'short_model_2000',
 'short_model_2002',
 'short_model_2006',
 'short_model_2007',
 'short_model_2008',
 'short_model_2009',
 'short_model_2012',
 'short_model_2015',
 'short_model_2016',
 'short_model_2017',
 'short_model_2018',
 'short_model_2021',
 'short_model_210',
 'short_model_240',
 'short_model_2500',
 'short_model_2500hd',
 'short_model_280zx',
 'short_model_3',
 'short_model_3-series',
 'short_model_3-window',
 'short_model_3.2',
 'short_model_300',
 'short_model_300-series',
 'short_mode

In [105]:
print(df_prepared.shape)

(9964, 1483)


In [106]:
df_prepared.to_csv('data/vehicles_dataset_prepared.csv')

# Modeling

Сформируем датасет для обучения, инициализируем фичи и целевую переменную (price_category), положим их в соответствующие переменные, разделим данные на тестовую и тренировочные части

In [107]:
df_modeling = df_prepared.drop(['price', 'id'], axis=1)
x = df_modeling.drop(['price_category'], axis=1)
y = df_modeling['price_category']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)

## Логистическая регрессия

In [108]:
log = LogisticRegression()
log.fit(x_train, y_train)
print(accuracy_score(y_train, log.predict(x_train)))
print(confusion_matrix(y_train, log.predict(x_train)))

0.937338686550043
[[2307    1   88]
 [   0 2135  109]
 [  62  177 2095]]


C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [109]:
log.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [110]:
log = LogisticRegression(random_state = 42, solver = 'newton-cg', C= 4.1)
log.fit(x_train, y_train)
print(accuracy_score(y_train, log.predict(x_train)))
print(confusion_matrix(y_train, log.predict(x_train)))

0.9728993404072268
[[2364    0   32]
 [   0 2196   48]
 [  23   86 2225]]


## Случайный лес

In [111]:
rf_clf = RandomForestClassifier(random_state = 50)
rf_clf.fit(x_train, y_train)

RandomForestClassifier(random_state=50)

In [112]:
pred_rf = rf_clf.predict(x_test)
print(accuracy_score(y_train, rf_clf.predict(x_train)))
print(confusion_matrix(y_train, rf_clf.predict(x_train)))

1.0
[[2396    0    0]
 [   0 2244    0]
 [   0    0 2334]]


In [113]:
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 50,
 'verbose': 0,
 'warm_start': False}

In [114]:
rf_clf = RandomForestClassifier(random_state = 42, max_features = 'log2', min_samples_split = 3)
rf_clf.fit(x_train, y_train)
pred_rf = rf_clf.predict(x_test)
print(accuracy_score(y_train, rf_clf.predict(x_train)))
print(confusion_matrix(y_train, rf_clf.predict(x_train)))

1.0
[[2396    0    0]
 [   0 2244    0]
 [   0    0 2334]]


## Многослойный персептрон


In [115]:
mlp = MLPClassifier( random_state=42, max_iter=500)
mlp.fit(x_train, y_train)
print(accuracy_score(y_train, mlp.predict(x_train)))
print(confusion_matrix(y_train, mlp.predict(x_train)))

1.0
[[2396    0    0]
 [   0 2244    0]
 [   0    0 2334]]


In [116]:
mlp.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 42,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [117]:
mlp = MLPClassifier( random_state=42, max_iter=600, activation = 'tanh', hidden_layer_sizes = (100,24))
mlp.fit(x_train, y_train)
print(accuracy_score(y_train, mlp.predict(x_train)))
print(confusion_matrix(y_train, mlp.predict(x_train)))

0.9911098365357041
[[2385    0   11]
 [   0 2225   19]
 [  13   19 2302]]


## Проверка качества с помощью кросс-валидации

Проверим предсказательную способность моделей логистической регрессии, случайного леса и многослойного персептрона с помощью кросс-валидации.

In [118]:
cross_validate(log, x, y, cv=5)

{'fit_time': array([8.81101704, 8.65687776, 8.89007998, 9.30046225, 9.06024337]),
 'score_time': array([0.02602339, 0.02505064, 0.02503204, 0.0240221 , 0.02402139]),
 'test_score': array([0.9061716 , 0.91369794, 0.91169092, 0.91570497, 0.91666667])}

In [119]:
cv_score_log=cross_val_score(log, x, y, cv=5)
cv_score_log.mean(), cv_score_log.std()

(0.9127864191336345, 0.0037245420304408074)

In [120]:
cross_validate(rf_clf, x, y, cv=5)

{'fit_time': array([3.53421855, 3.61328888, 3.6313045 , 3.60427952, 3.54621792]),
 'score_time': array([0.11109805, 0.1141119 , 0.11510468, 0.11510515, 0.10909963]),
 'test_score': array([0.97390868, 0.96086302, 0.96738585, 0.96738585, 0.97238956])}

In [121]:
cv_score_rf=cross_val_score(rf_clf, x, y, cv=5)
cv_score_rf.mean(), cv_score_rf.std()

(0.968386592027921, 0.0045852871879194495)

In [122]:
cross_validate(mlp, x, y, cv=5)

{'fit_time': array([49.73124981, 44.43843365, 46.93982577, 52.55717802, 41.88711238]),
 'score_time': array([0.03003597, 0.03203821, 0.0330286 , 0.03503203, 0.03003621]),
 'test_score': array([0.9347717 , 0.92574009, 0.93125941, 0.93075765, 0.93072289])}

In [123]:
cv_score_mlp=cross_val_score(mlp, x, y, cv=5)
cv_score_mlp.mean(), cv_score_mlp.std()

(0.9306503485089379, 0.0028804135589971572)

По результатам кросс-валидации лучшей моделью является модель случайного леса. Посчитаем значение модели случайного леса для тестовой выборки.

In [124]:
predicted_test_rf = rf_clf.predict(x_test)
print(accuracy_score(y_test, predicted_test_rf))

0.9775919732441471


# Results

По результатам моделирования лучший результат показала модель случайного леса. Обучим эту модель на всём датасете.

In [125]:
rf_clf.fit(x,y)

RandomForestClassifier(max_features='log2', min_samples_split=3,
                       random_state=42)

In [126]:
from io import open_code
filename='data/final_model.pickle'

with open(filename, 'wb') as file:
    pickle.dump(rf_clf, file)